In [1]:
# 회원데이터 가져오기
import pandas as pd
import cx_Oracle

# Oracle 데이터베이스 연결 설정
dsn_tns = cx_Oracle.makedsn('DB URL', 'PORT번호', service_name='XE')
conn = cx_Oracle.connect(user='DB사용자이름', password='DB PASSWORD', dsn=dsn_tns)

# SQL 쿼리로 사용자 테이블 데이터 가져오기
query = 'SELECT * FROM TBL_USER'
df_user = pd.read_sql(query, con=connection)

# CSV 파일로 저장
csv_file_path = 'user_data1.csv'
df_user.to_csv(csv_file_path, index=False, encoding='EUC-KR')

print(f"파일저장 {csv_file_path}")

# 연결 종료
connection.close()

저장 user_data1.csv


C:\Users\USER\AppData\Local\Temp\ipykernel_12272\3862591428.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_user = pd.read_sql(query, con=connection)


In [6]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 사용자 데이터 로드
user_file_path = '저장된 유저 데이터'
df_user = pd.read_csv(user_file_path, encoding='EUC-KR')

# 전처리된 정책 데이터 로드
policy_file_path = '전처리된 정책데이터'
df_policy = pd.read_csv(policy_file_path)

# 불용어 리스트
stopwords = ['을', '를', '이', '가', '은', '는', '에', '의', '와', '한', '에서', '으로', '및', '또는']

# 텍스트 정규화 및 토큰화 함수
def preprocess_text_simple(text):
    if not isinstance(text, str):
        return ''
    text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords]
    return ' '.join(tokens)

# 텍스트 전처리 적용
df_policy['policy_desc_processed'] = df_policy['policy_desc'].apply(preprocess_text_simple)
df_policy['support_content_processed'] = df_policy['support_content'].apply(preprocess_text_simple)

# TF-IDF 벡터화
tfidf_vectorizer_desc = TfidfVectorizer()
tfidf_matrix_desc = tfidf_vectorizer_desc.fit_transform(df_policy['policy_desc_processed'])

tfidf_vectorizer_support = TfidfVectorizer()
tfidf_matrix_support = tfidf_vectorizer_support.fit_transform(df_policy['support_content_processed'])

# 코사인 유사도 계산
cosine_sim_desc = cosine_similarity(tfidf_matrix_desc, tfidf_matrix_desc)
cosine_sim_support = cosine_similarity(tfidf_matrix_support, tfidf_matrix_support)

# 정책 데이터를 코드로 매핑
df_policy['region_code'] = df_policy['org_code']
df_policy['policy_interest_code'] = df_policy['policy_field_code']

# 사용자 기반 데이터 필터링 및 추천
def get_recommendations(user_profile, df, tfidf_vectorizer_desc, tfidf_vectorizer_support, num_recommendations=5):
    user_region = user_profile['USER_REGION']
    user_region_code = user_profile['USER_REGION_CODE']
    user_policy_interest_code = user_profile['USER_POLICY_INTEREST_CODE']
    
    # 사용자 지역을 포함한 텍스트로 유사도 계산
    user_region_tfidf_desc = tfidf_vectorizer_desc.transform([user_region])
    user_region_tfidf_support = tfidf_vectorizer_support.transform([user_region])
    
    region_cosine_sim_desc = cosine_similarity(user_region_tfidf_desc, tfidf_matrix_desc).flatten()
    region_cosine_sim_support = cosine_similarity(user_region_tfidf_support, tfidf_matrix_support).flatten()

    # 필터링
    filtered_df = df[
        (df['region_code'] == user_region_code) &
        (df['policy_interest_code'] == user_policy_interest_code)
    ]
    
    # 필터링된 데이터 확인
    print(f"Filtered policies for region code {user_region_code} and policy interest code {user_policy_interest_code}:")
    print(filtered_df[['policy_id', 'org_code', 'policy_name', 'policy_desc']])
    
    if filtered_df.empty:
        return [], []

    tfidf_matrix_desc_filtered = tfidf_vectorizer_desc.transform(filtered_df['policy_desc_processed'])
    tfidf_matrix_support_filtered = tfidf_vectorizer_support.transform(filtered_df['support_content_processed'])

    # 필터링된 데이터 내에서 유사도 계산
    cosine_sim_desc_filtered = cosine_similarity(tfidf_matrix_desc_filtered, tfidf_matrix_desc_filtered)
    cosine_sim_support_filtered = cosine_similarity(tfidf_matrix_support_filtered, tfidf_matrix_support_filtered)

    recommendations_desc = []
    recommendations_support = []

    for idx in range(len(filtered_df)):
        desc_sim_scores = list(enumerate(cosine_sim_desc_filtered[idx] * region_cosine_sim_desc[filtered_df.index]))
        desc_sim_scores = sorted(desc_sim_scores, key=lambda x: x[1], reverse=True)
        desc_sim_scores = desc_sim_scores[:num_recommendations]
        desc_policy_indices = [i[0] for i in desc_sim_scores]
        recommendations_desc.extend(filtered_df.iloc[desc_policy_indices]['policy_name'])
        
        support_sim_scores = list(enumerate(cosine_sim_support_filtered[idx] * region_cosine_sim_support[filtered_df.index]))
        support_sim_scores = sorted(support_sim_scores, key=lambda x: x[1], reverse=True)
        support_sim_scores = support_sim_scores[:num_recommendations]
        support_policy_indices = [i[0] for i in support_sim_scores]
        recommendations_support.extend(filtered_df.iloc[support_policy_indices]['policy_name'])
    
    recommendations_desc = list(set(recommendations_desc))
    recommendations_support = list(set(recommendations_support))
    
    return recommendations_desc[:5], recommendations_support[:5]

# 사용자 프로필 로드 및 추천 생성
user_profile = df_user[df_user['USER_ID'] == 'pam'].iloc[0]
recommendations_desc, recommendations_support = get_recommendations(user_profile, df_policy, tfidf_vectorizer_desc, tfidf_vectorizer_support, num_recommendations=10)

# 결과 출력
print("정책 설명 기반 추천:")
print(recommendations_desc)
print("\n지원 내용 기반 추천:")
print(recommendations_support)

Filtered policies for region code 3002012 and policy interest code 23020:
           policy_id  org_code            policy_name  \
1164  R2024041621859   3002012  전세보증금반환보증 보증료 지원사업 안내   
1246  R2024030420263   3002012    청년월세 한시 특별지원 사업(2차)   

                                            policy_desc  
1164                          전세보증금반환보증 보증료 지원(최대 30만원)  
1246  경제적 어려움을 겪는 청년층의 주거비 부담을 덜어드리기 위해 12개월 분의 월세를 ...  
정책 설명 기반 추천:
['청년월세 한시 특별지원 사업(2차)', '전세보증금반환보증 보증료 지원사업 안내']

지원 내용 기반 추천:
['청년월세 한시 특별지원 사업(2차)', '전세보증금반환보증 보증료 지원사업 안내']
